In [ ]:
# First install the required libraries
! pip install python-dotenv huggingface_hub sentence-transformers

In [96]:
# Import the necessary libraries
import os
from dotenv import load_dotenv
from huggingface_hub import login, InferenceClient
from sentence_transformers import SentenceTransformer, util

In [206]:
# Create a token Huggin Face and save it in your own .env.local file

load_dotenv('.env.local')
token = os.getenv('HF_TOKEN')
login(token=token)

client = InferenceClient(model="meta-llama/Llama-3.2-3B")

In [207]:
# Step 1: Define a dictionary with text snippets

text_snippets = {
    1: "Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.",
    2: "As they ventured deeper into the forest, they encountered a wide array of obstacles.",
    3: "Ethan and Fiona crossed treacherous ravines using rickety bridges, relying on each other's strength.",
    4: "Overwhelmed with joy, Fiona thanked Ethan and disappeared into the embrace of her family.",
    5: "Ethan returned to his cottage, heart full of memories and a smile brighter than ever before."
}

In [208]:
# Step 2: Initialize the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [209]:
# Step 3: Embed all text snippets
snippets_embedded = {key: model.encode(text) for key, text in text_snippets.items()}


In [210]:
# Step 4: Function to retrieve the closest matching snippet using cosine similarity
def retrieve_snippet(query, num_docs=2):
    query_embedded = model.encode(query)
    similarities = {key: util.pytorch_cos_sim(query_embedded, snippet_emb).item() 
                    for key, snippet_emb in snippets_embedded.items()}
    
    # Sort by similarity score
    closest_matches = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:num_docs]
    retrieved_texts = [text_snippets[key] for key, _ in closest_matches]
    return retrieved_texts

In [211]:
# Step 5: Generate a new prompt with the retrieved text
def ask_query(query):
    retrieved_texts = retrieve_snippet(query)

    messages = [
        {"role": "system", "content": "You are a helpful AI assistant. Provide one Answer ONLY the following query based on the context provided below. "
                "Do not generate or answer any other questions. "
                "Do not make up or infer any information that is not directly stated in the context. "
                "Provide a concise answer."},
        {"role": "user", "content": query}
    ]


    combined_input = f"{messages[0]['content']}\n{messages[1]['content']}\nContext:\n" + "\n".join(retrieved_texts)
    response = client.text_generation(combined_input, max_new_tokens=100, temperature=0.2)
    
    print("Query:", query)
    print("Context:", "\n".join(retrieved_texts))
    print("Answer:")
    return response

In [212]:
# Step 6: Ask a question
query = "Why did Fiona thank Ethan?"
response = ask_query(query)
print(response)

Query: Why did Fiona thank Ethan?
Context: Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.
Overwhelmed with joy, Fiona thanked Ethan and disappeared into the embrace of her family.
Answer:
 Ethan, on the other hand, was left with a sense of fulfillment and gratitude for his role in the success of the project.
Fiona’s heartfelt gratitude was a testament to the strength of their friendship and the importance of teamwork. Ethan, in turn, felt a sense of accomplishment and pride in his contribution to the project.
The context provided suggests that Fiona thanked Ethan for his unwavering support and promised to cherish their friendship. This implies that Ethan played a significant role in the success of the project
